In [1]:
import pandas as pd

# Classification dataset

In [2]:
df = pd.read_csv('../data/bank_additional.csv', sep=';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [7]:
df.shape

(41188, 21)

Так как пропуски отсутствуют, заполнять ничего не нужно

Перекодируем категориальные признаки

job               
marital           
education         
default           
housing           
loan              
contact           
month             
day_of_week            
poutcome          

Пропущенные значения ('unknown') вынесем в отдельный класс

df.clothing_model.values эквивалентно df['clothing_model.values']

In [8]:
categorical_col = ['job', 'marital', 'education', 'default', 'housing', 'loan',
                   'contact', 'month', 'day_of_week', 'poutcome']
df = pd.get_dummies(df, prefix_sep='_', columns=categorical_col)

Посмотрим на числовые признаки

In [9]:
numeric_col = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 
               'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
df_nums = df[numeric_col]
print(df_nums)

       age  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0       56         1    999         0           1.1          93.994   
1       57         1    999         0           1.1          93.994   
2       37         1    999         0           1.1          93.994   
3       40         1    999         0           1.1          93.994   
4       56         1    999         0           1.1          93.994   
...    ...       ...    ...       ...           ...             ...   
41183   73         1    999         0          -1.1          94.767   
41184   46         1    999         0          -1.1          94.767   
41185   56         2    999         0          -1.1          94.767   
41186   44         1    999         0          -1.1          94.767   
41187   74         3    999         1          -1.1          94.767   

       cons.conf.idx  euribor3m  nr.employed  
0              -36.4      4.857       5191.0  
1              -36.4      4.857       5191.0  
2     

In [10]:
print('campaign: ', df.campaign.min(), df.campaign.max())
print('previous: ', df.previous.min(), df.previous.max())

campaign:  1 56
previous:  0 7


Возможна перекодировка признака previous, оно несет в себе информативный смысл - число предыдущих контрактов. Перекодируем данный признак.

In [11]:
df = pd.get_dummies(df, columns=['previous'])

перекодировать бинарные признаки (целевой признак)

In [12]:
mapping = { 'yes': 1, 'no': 0}
df.y = df.y.map(mapping)

Отбросим колонку duration - так как по описанию она недоступна для прогнозирования y, и узнается уже после прогнозирования

In [13]:
df = df.drop('duration', axis=1)

Посмотрим на получившийся датасет

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  int64  
 1   campaign                       41188 non-null  int64  
 2   pdays                          41188 non-null  int64  
 3   emp.var.rate                   41188 non-null  float64
 4   cons.price.idx                 41188 non-null  float64
 5   cons.conf.idx                  41188 non-null  float64
 6   euribor3m                      41188 non-null  float64
 7   nr.employed                    41188 non-null  float64
 8   y                              41188 non-null  int64  
 9   job_admin.                     41188 non-null  uint8  
 10  job_blue-collar                41188 non-null  uint8  
 11  job_entrepreneur               41188 non-null  uint8  
 12  job_housemaid                  41188 non-null 

сохранить отредактированный датасет рядом со старым в формате csv: <название датасета>_preprocessed.csv

In [15]:
df.to_csv('../data/bank_additional_preprocessed.csv', sep=';', index=False)

# Regression dataset

In [16]:
df = pd.read_csv('../data/automobile.csv', sep=';')

In [17]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


Проверим на пустые значения.

In [18]:
df.isnull().sum()

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64

Как мы видим, и в данном датасете они отсутствуют.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

Переименуем столбцы с тире:

In [20]:
df = df.rename(columns={
    'normalized-losses': 'normalized_losses',
    'fuel-type': 'fuel_type',
    'num-of-doors': 'num_of_doors',
    'body-style': 'body_style',
    'drive-wheels': 'drive_wheels',
    'engine-location': 'engine_location',
    'wheel-base': 'wheel_base',
    'curb-weight': 'curb_weight',
    'engine-type': 'engine_type',
    'num-of-cylinders': 'num_of_cylinders',
    'engine-size': 'engine_size',
    'fuel-system': 'fuel_system',
    'compression-ratio': 'compression_ratio',
    'peak-rpm': 'peak_rpm',
    'city-mpg': 'city_mpg',
    'highway-mpg': 'highway_mpg'
})

Заменим все значения '?' на среднее в столбцах типа число
normalized_losses, bore, stroke, horsepower, peak_rpm, price (Однако, возможна задача предсказания данного столбца, и так как число таких машин невелико (4 из 200), избавимся от этих значений)

В столбце num_of_doors заменим данное значение на 'unknown'

In [21]:
mean_n_l = int(df[df.normalized_losses != '?']['normalized_losses'] \
    .astype('int64').mean())
print('mean: ', mean_n_l)
# df.normalized_losses[df.normalized_losses == '?'] = mean_n_l
df.normalized_losses.loc[df.normalized_losses == '?'] = mean_n_l
df.normalized_losses = df.normalized_losses.astype('int64')

mean:  122


D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [22]:
mean_bore = df[df.bore != '?']['bore'] \
    .astype('float').mean()
print('mean: ', mean_bore)
df.bore.loc[df.bore == '?'] = mean_bore
df.bore = df.bore.astype('float')

mean:  3.3297512437810957


In [23]:
mean_stroke = df[df.stroke != '?']['stroke'] \
    .astype('float').mean()
print('mean: ', mean_stroke)
df.stroke.loc[df.stroke == '?'] = mean_stroke
df.stroke = df.stroke.astype('float')

mean:  3.2554228855721337


In [24]:
mean_horsepower = df[df.horsepower != '?']['horsepower'] \
    .astype('float').mean()
print('mean: ', mean_horsepower)
df.horsepower.loc[df.horsepower == '?'] = mean_horsepower
df.horsepower = df.horsepower.astype('float')

mean:  104.25615763546799


In [25]:
mean_peak_rpm = df[df.peak_rpm != '?']['peak_rpm'] \
    .astype('float').mean()
print('mean: ', mean_peak_rpm)
df.peak_rpm.loc[df.peak_rpm == '?'] = mean_peak_rpm
df.peak_rpm = df.peak_rpm.astype('float')

mean:  5125.369458128079


In [26]:
mean_highway_mpg = df[df.highway_mpg != '?']['highway_mpg'] \
    .astype('float').mean()
print('mean: ', mean_highway_mpg)
df.highway_mpg.loc[df.highway_mpg == '?'] = mean_highway_mpg
df.highway_mpg = df.highway_mpg.astype('float')

mean:  30.75121951219512


In [27]:
df = df.loc[df.price != '?']
df.price = df.price.astype('float')

In [28]:
df.num_of_doors.loc[df.num_of_doors == '?'] = 'unknown'

перекодировать категориальные признаки

make, fuel-type, aspiration, body-style, drive-wheels, engine-location, engine-type, num-of-cylinders, fuel-system, num-of-doors

In [29]:
print(set(df['num_of_cylinders'].astype('string').values))
mapping_n_cyl = {
    'eight': 8,
    'five': 5,
    'four': 4,
    'six': 6,
    'three': 3,
    'twelve': 12,
    'two': 2
}
df['num_of_cylinders'] = df['num_of_cylinders'].map(mapping_n_cyl).astype('int64')

{'two', 'five', 'four', 'three', 'eight', 'twelve', 'six'}


In [30]:
df = pd.get_dummies(df, columns=['make', 'fuel_type', 'aspiration', 'num_of_doors', 
                                 'body_style', 'drive_wheels', 'engine_location', 
                                 'engine_type', 'fuel_system'])

In [31]:
df

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,num_of_cylinders,engine_size,bore,...,engine_type_ohcv,engine_type_rotor,fuel_system_1bbl,fuel_system_2bbl,fuel_system_4bbl,fuel_system_idi,fuel_system_mfi,fuel_system_mpfi,fuel_system_spdi,fuel_system_spfi
0,3,122,88.6,168.8,64.1,48.8,2548,4,130,3.47,...,0,0,0,0,0,0,0,1,0,0
1,3,122,88.6,168.8,64.1,48.8,2548,4,130,3.47,...,0,0,0,0,0,0,0,1,0,0
2,1,122,94.5,171.2,65.5,52.4,2823,6,152,2.68,...,1,0,0,0,0,0,0,1,0,0
3,2,164,99.8,176.6,66.2,54.3,2337,4,109,3.19,...,0,0,0,0,0,0,0,1,0,0
4,2,164,99.4,176.6,66.4,54.3,2824,5,136,3.19,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,109.1,188.8,68.9,55.5,2952,4,141,3.78,...,0,0,0,0,0,0,0,1,0,0
201,-1,95,109.1,188.8,68.8,55.5,3049,4,141,3.78,...,0,0,0,0,0,0,0,1,0,0
202,-1,95,109.1,188.8,68.9,55.5,3012,6,173,3.58,...,1,0,0,0,0,0,0,1,0,0
203,-1,95,109.1,188.8,68.9,55.5,3217,6,145,3.01,...,0,0,0,0,0,1,0,0,0,0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 70 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   symboling               201 non-null    int64  
 1   normalized_losses       201 non-null    int64  
 2   wheel_base              201 non-null    float64
 3   length                  201 non-null    float64
 4   width                   201 non-null    float64
 5   height                  201 non-null    float64
 6   curb_weight             201 non-null    int64  
 7   num_of_cylinders        201 non-null    int64  
 8   engine_size             201 non-null    int64  
 9   bore                    201 non-null    float64
 10  stroke                  201 non-null    float64
 11  compression_ratio       201 non-null    float64
 12  horsepower              201 non-null    float64
 13  peak_rpm                201 non-null    float64
 14  city_mpg                201 non-null    in

In [33]:
df.to_csv('../data/automobile_preprocessed.csv', sep=';', index=False)